In [ ]:
# Importing the necessary libraries
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
from statistics import mean
import datetime as datetime

In [ ]:
# Data pre-processing
transform_train = transforms.Compose([transforms.Resize((70,70)), # Resizing the dataset samples to 70x70
                                transforms.RandomHorizontalFlip(), # Randomly flips the data samples horizontally
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), # Normalizing the dataset
                                                     (0.5, 0.5, 0.5))])

transform_test = transforms.Compose([transforms.Resize((70,70)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.5, 0.5, 0.5), 
                                                     (0.5, 0.5, 0.5))])                                                    

# Downloading and loading the dataset into the workspace
train = torchvision.datasets.CIFAR10(root='./data',train=True, transform=transform_train, download=True)
test = torchvision.datasets.CIFAR10(root='./data',train=False, transform=transform_test, download=True)
train_loader = torch.utils.data.DataLoader(dataset=train, batch_size=32, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test, batch_size=32, shuffle=False)
cuda = torch.device('cuda') # training the model on a GPU

In [ ]:
# Structure of the ResNet-34 architecture
class Bottleneck(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super().__init__()
        self.conv1 = nn.Conv2d(inplanes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        identity = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            identity = self.downsample(x)

        out += identity
        out = self.relu(out)
        return out

class ResNet34(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super().__init__()    
        self.inplanes = 64
        self.conv1 = nn.Conv2d(3, self.inplanes, kernel_size=7, stride=2, padding=3, bias=False)
        self.bn1 = nn.BatchNorm2d(self.inplanes)
        self.relu = nn.ReLU(inplace=True)
        self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        
        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 , num_classes)


    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None     
        if stride != 1 or self.inplanes != planes:
            downsample = nn.Sequential(nn.Conv2d(self.inplanes, planes, 1, stride, bias=False),
                                       nn.BatchNorm2d(planes))
        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        
        self.inplanes = planes
        
        for _ in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)
    
    
    def forward(self, x):
        x = self.conv1(x)          
        x = self.bn1(x)
        x = self.relu(x)
        x = self.maxpool(x)         

        x = self.layer1(x)         
        x = self.layer2(x)          
        x = self.layer3(x)         
        x = self.layer4(x)          

        x = self.avgpool(x)        
        x = torch.flatten(x, 1)     
        x = self.fc(x)

        return x

def resnet34():
    layers=[3, 4, 6, 3]
    model = ResNet34(Bottleneck, layers)
    return model

model = resnet34()
print(model)

In [ ]:
model = model.cuda() # Using CUDA enabled GPU for training the model
loss = nn.CrossEntropyLoss() # Using Cross Entropy as Loss function
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9) # Setting the optimizer for training the model
cost = 0
epochs = 20 # Setting the number of epochs for which the model is to be trained

iterations = []
train_losses = []
test_losses = []
train_acc = []
test_acc = []

# Training the model
for epoch in range(epochs):
    model.train()
    correct = 0
    for X, Y in train_loader:
        X = X.to(cuda)
        Y = Y.to(cuda)
        optimizer.zero_grad()
        hypo = model(X)
        cost = loss(hypo, Y)
        cost.backward()
        optimizer.step()
        prediction = hypo.data.max(1)[1]
        correct += prediction.eq(Y.data).sum()

    model.eval()
    correct2 = 0
    for data, target in test_loader:
        data = data.to(cuda)
        target = target.to(cuda)
        output = model(data)
        cost2 = loss(output, target)
        prediction = output.data.max(1)[1]
        correct2 += prediction.eq(target.data).sum()

    iterations.append(epoch)
    train_losses.append(cost.tolist())
    test_losses.append(cost2.tolist())
    train_acc.append((100*correct/len(train_loader.dataset)).tolist())
    test_acc.append((100*correct2/len(test_loader.dataset)).tolist())
    print("Epoch : {:>4} / cost : {:>.9}".format(epoch + 1, cost))
    print('Train set Accuracy: {:.2f}%'.format(100. * correct / len(train_loader.dataset)))
    print('Test set Accuracy: {:.2f}%'.format(100. * correct2 / len(test_loader.dataset)))
    timestamp = datetime.datetime.now()
    print("Date, Time stamp: ", timestamp)

In [ ]:
# Train Accuracy vs Validation accuracy plot
plt.figure(figsize=(10, 7))
plt.plot(train_acc, color='green', label='train accuracy')
plt.plot(test_acc, color='blue', label='validataion accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()

# Train Loss vs Validation Loss plot
plt.figure(figsize=(10, 7))
plt.plot(train_losses, color='orange', label='train loss')
plt.plot(test_losses, color='red', label='validataion loss')
plt.legend()
plt.savefig('loss.png')
plt.show()

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # Utilizng CUDA enabled GPU
print(device)
classes = ['airplane','automobile','bird', # Classes of the CIFAR-10 dataset
           'cat','deer','dog','frog',
           'horse','ship','truck']

# Building the heatmap of the correctly classified and misclassified data samples
heatmap = pd.DataFrame(data=0,index=classes,columns=classes)
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(16):
            true_label = labels[i].item()
            predicted_label = predicted[i].item()
            heatmap.iloc[true_label,predicted_label] += 1
_, ax = plt.subplots(figsize=(10, 8))
ax = sns.heatmap(heatmap, annot=True, fmt='d',cmap='YlGnBu')
figure = ax.get_figure()    
figure.savefig('heatmap.png', dpi=400)
plt.show()

In [ ]:
# Saving the model
torch.save(model,'ResNet34_CIFAR10.pth')